# Network creationg - SNet, SNetF, SNetT, UserNet


## Importing data

In [1]:
%pip install pandas
import pandas as pd

%pip install pickle
import pickle

%pip install networkx
import networkx as nx

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle


Note: you may need to restart the kernel to use updated packages.


In [2]:
with open("data/comments_cleaned", 'rb') as file:
    comments = pickle.load(file)
    
with open("data/submissions_cleaned", 'rb') as file:
    submissions = pickle.load(file)

In [3]:
submissions

,submissions_id,url,permalink,author,created_utc,subreddit,subreddit_id,num_comments,score,over_18,distinguished,domain,stickied,locked,hide_score,id
0,648oo,http://www.ignorancedenied.com/viewthread.php?...,/r/reddit.com/comments/648oo/brain_disease_is_...,DITUS,1199145615,reddit.com,t5_6,1,0,False,<NA>,ignorancedenied.com,False,False,False,0
1,648op,http://www.flascience.org/wp/?p=363,/r/science/comments/648op/three_more_florida_c...,rmuser,1199145634,science,t5_mouw,5,20,False,<NA>,flascience.org,False,False,False,1
2,648or,http://hosted.ap.org/dynamic/stories/O/ODD_SHO...,/r/reddit.com/comments/648or/nude_couple_grapp...,zorno,1199145709,reddit.com,t5_6,1,3,False,<NA>,hosted.ap.org,False,False,False,2
3,648os,http://www.sltrib.com/opinion/ci_7846101?sourc...,/r/politics/comments/648os/apparently_bushs_pr...,rmuser,1199145735,politics,t5_2cneq,2,0,False,<NA>,sltrib.com,False,False,False,3
4,648ot,http://hosted.ap.org/dynamic/stories/O/ODD_RAR...,/r/reddit.com/comments/648ot/diners_find_rare_...,zorno,1199145735,reddit.com,t5_6,0,0,False,<NA>,hosted.ap.org,False,False,False,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2044805,7mq3n,http://ventaboutsports.blogspot.com/2008/12/so...,/r/funny/comments/7mq3n/some_extremely_corny_j...,themightymidget,1230767909,funny,t5_2qh33,0,1,False,<NA>,ventaboutsports.blogspot.com,False,False,False,2044805
2044806,7mq3o,http://www.pbs.org/mormons/etc/genealogy.html,/r/news/comments/7mq3o/pbs_looks_at_the_massiv...,Tom22,1230767926,news,t5_2qh3l,0,0,False,<NA>,pbs.org,False,False,False,2044806
2044807,7mq3q,http://www.narutogames.biz,/r/reddit.com/comments/7mq3q/naruto_games/,bixiebix,1230767937,reddit.com,t5_6,7,1,False,<NA>,narutogames.biz,False,False,False,2044807
2044808,7mq3r,http://www.youtube.com/watch?v=gdQH1CI4LHY&amp...,/r/politics/comments/7mq3r/ron_paul_on_recent_...,middkidd,1230767963,politics,t5_2cneq,3,1,False,<NA>,youtube.com,False,False,False,2044808


In [4]:
comments

,comments_id,author,link_id,parent_id,created_utc,subreddit,subreddit_id,score,distinguished,gilded,controversiality,id
0,c02s9s6,Haven,648oh,t1_c02s9rv,1199145604,reddit.com,t5_6,4,<NA>,0,0,0
1,c02s9s8,lilmiss2,648oh,t1_c02s9rv,1199145620,reddit.com,t5_6,2,<NA>,0,0,1
2,c02s9sc,EverybodysAnAsshole,648et,t1_c02s976,1199145644,reddit.com,t5_6,2,<NA>,0,0,2
3,c02s9sd,generalk,647yd,t1_c02s8md,1199145647,programming,t5_2fwo,13,<NA>,0,0,3
4,c02s9se,seeker135,6483n,t3_6483n,1199145650,politics,t5_2cneq,4,<NA>,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...
4873684,c06vwud,CommodoreGuff,7k1l5,t1_c06vpzj,1229579674,programming,t5_2fwo,1,<NA>,0,0,4873684
4873685,c06vwue,wolfzero,7k4if,t1_c06vs7l,1229579675,technology,t5_2qh16,4,<NA>,0,0,4873685
4873686,c06vwug,Morgin_Black,7k3w5,t3_7k3w5,1229579679,comics,t5_2qh0s,0,<NA>,0,0,4873686
4873687,c06vwui,onezerozeroone,7k2bc,t1_c06vrvz,1229579685,atheism,t5_2qh2p,1,<NA>,0,0,4873687


## Creating networks

### General 

    - Create 3 different networks.
    - Nodes are subreddits.
    - Edges are people who have been active on two.
    - Weights are formed like a**n, where n is number of monthers (1-12) and a is 0..1

### 1. SNet - Subreddit network 

All subreddits and edges formed from comments and submissions.

In [38]:
all_subreddits = pd.concat([submissions[["subreddit"]], comments[["subreddit"]]], ignore_index=True)
all_subreddits.drop_duplicates(subset=['subreddit'], inplace=True, ignore_index=True)
print("Check number of nodes with the value from static analysis")
print("Number of subreddits: ", all_subreddits.shape[0])
%store -r NUMBER_OF_SUBREDDITS 
assert(all_subreddits.shape[0] == NUMBER_OF_SUBREDDITS)



Check number of nodes with the value from static analysis
Number of subreddits:  4359


In [61]:
SNet = nx.Graph()
# SNet.add_nodes_from()
print("Number of subreddits: ", all_subreddits["subreddit"].shape[0])
print(SNet.size())
all_subreddits["subreddit"]

Number of subreddits:  4359
0


0                reddit.com
1                   science
2                  politics
3                        ja
4                    sports
               ...         
4354             FootFetish
4355               Spanking
4356         JailbaitPhotos
4357                 aragon
4358    maculardegeneration
Name: subreddit, Length: 4359, dtype: string